In [41]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import matplotlib
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

In [36]:
test = pd.read_csv('test.csv',parse_dates=['date_created','user_date_created','month_created'])
train = pd.read_csv('train.csv',parse_dates=['date_created','user_date_created'])

In [39]:
test.head()
type(test.loc[0,'month_created'])

pandas._libs.tslib.Timestamp

In [6]:
pp.ProfileReport(test)

Number of variables,11
Number of observations,252099
Total Missing (%),10.0%
Total size in memory,21.2 MiB
Average record size in memory,88.0 B
Numeric,4
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,1


In [7]:
pp.ProfileReport(train)

Number of variables,11
Number of observations,1281499
Total Missing (%),3.2%
Total size in memory,99.0 MiB
Average record size in memory,81.0 B
Numeric,4
Categorical,5
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [9]:
train.shape[1],train.shape[1]

(11, 11)

In [11]:
train.corr()

,comment_id,company_id,user_id,product_id,rating,is_fake
comment_id,1.000000,0.007737,0.945726,0.802164,0.025298,0.096783
company_id,0.007737,1.000000,0.006817,0.007686,0.009180,0.020309
user_id,0.945726,0.006817,1.000000,0.753244,0.035457,0.120007
product_id,0.802164,0.007686,0.753244,1.000000,0.000280,0.103157
rating,0.025298,0.009180,0.035457,0.000280,1.000000,0.026522
is_fake,0.096783,0.020309,0.120007,0.103157,0.026522,1.000000


In [12]:
test.corr()

,comment_id,company_id,product_id,rating,user_id
comment_id,1.000000,-0.008404,0.753073,0.062585,0.965797
company_id,-0.008404,1.000000,-0.006895,0.008105,-0.009369
product_id,0.753073,-0.006895,1.000000,0.004563,0.718010
rating,0.062585,0.008105,0.004563,1.000000,0.070159
user_id,0.965797,-0.009369,0.718010,0.070159,1.000000


In [13]:
 test['month_created']   

0         2018-03
1         2017-12
2         2017-11
3         2017-11
4         2017-11
5         2017-11
6         2017-11
7         2017-11
8         2017-11
9         2017-11
10        2017-11
11        2017-11
12        2018-02
13        2017-11
14        2017-11
15        2017-11
16        2017-11
17        2018-01
18        2017-11
19        2017-11
20        2017-11
21        2018-02
22        2017-12
23        2017-11
24        2017-11
25        2017-12
26        2017-12
27        2017-11
28        2017-11
29        2017-11
           ...   
252069        NaN
252070        NaN
252071        NaN
252072        NaN
252073        NaN
252074        NaN
252075        NaN
252076        NaN
252077        NaN
252078        NaN
252079        NaN
252080        NaN
252081        NaN
252082        NaN
252083        NaN
252084        NaN
252085        NaN
252086        NaN
252087        NaN
252088        NaN
252089        NaN
252090        NaN
252091        NaN
252092        NaN
252093    

In [19]:
test['month_created'].value_counts()

2018-02    23594
2018-01    16845
2018-03    16438
2017-12    15348
2017-11    10801
Name: month_created, dtype: int64

In [23]:
train['is_fake'].value_counts()

False    1276801
True        4698
Name: is_fake, dtype: int64

In [25]:
good_comment = train[train['is_fake']==False]

In [28]:
pp.ProfileReport(good_comment)

Number of variables,12
Number of observations,1276801
Total Missing (%),2.9%
Total size in memory,108.4 MiB
Average record size in memory,89.0 B
Numeric,4
Categorical,5
Boolean,0
Date,0
Text (Unique),0
Rejected,3


In [29]:
bad_comment = train[train['is_fake']==True]

In [30]:
pp.ProfileReport(bad_comment)

Number of variables,12
Number of observations,4698
Total Missing (%),1.8%
Total size in memory,408.4 KiB
Average record size in memory,89.0 B
Numeric,4
Categorical,4
Boolean,0
Date,0
Text (Unique),1
Rejected,3


In [40]:
train['len_com']= [ len(elem) for elem in train['comment'] ]

In [32]:
train.head()

,comment_id,company_id,user_id,product_id,comment,rating,date_created,user_date_created,user_ip,user_agent,is_fake,len_com
0,1,2169053,91933,6.0,компания не выполнила заказ без обьяснения пр...,1,2014-10-23 20:26:44,2010-02-03 21:03:22,178.92.70.220,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,False,51
1,2,709306,91933,2.0,"Впервые встретился такой професиональный, отве...",5,2016-12-10 08:38:27,2010-02-03 21:03:22,94.179.59.162,Mozilla/5.0 (Linux; Android 5.1.1; C6602 Build...,False,268
2,3,35287,91933,NaN,"Несерйозная, некоректная менеджер інтернет маг...",1,2017-02-11 15:55:14,2010-02-03 21:03:22,37.73.137.188,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False,105
3,4,1778322,91933,1.0,"Не рекомендую. Не отвечают, не выполняют заказы.",1,2017-06-26 18:00:45,2010-02-03 21:03:22,37.73.149.180,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False,48
4,5,2821533,91933,NaN,"На письма не отвечают, заказы не выполняют. Не...",1,2017-07-06 16:57:40,2010-02-03 21:03:22,37.73.221.126,Mozilla/5.0 (Linux; Android 6.0; ASUS_X008D Bu...,False,57


In [ ]:
from collections import defaultdict
def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat(
        [df[cat_cols] for df in df_list],
        axis=0
    ).fillna('').apply(
        lambda x: d[x.name].fit(x)
    ),
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(
            lambda x: d[x.name].transform(x))

In [ ]:
__encode_categorical(df_list=[df], cat_cols=['comment','date_created']+binary_cols)

In [44]:
all_text = pd.concat([train['comment'], test['comment']])

In [45]:
vectorizer_train = TfidfVectorizer(min_df = 5, max_features = 40000)
vectorizer_train.fit(all_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=40000, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [47]:
vectorizer_train.get_feature_names()

['00',
 '000',
 '001',
 '002',
 '003',
 '004',
 '005',
 '007',
 '008',
 '009',
 '00грн',
 '01',
 '010',
 '011',
 '012',
 '013',
 '016',
 '017',
 '02',
 '020',
 '03',
 '030',
 '04',
 '05',
 '050',
 '06',
 '063',
 '066',
 '067',
 '068',
 '07',
 '08',
 '09',
 '093',
 '095',
 '096',
 '097',
 '098',
 '099',
 '10',
 '100',
 '1000',
 '10000',
 '1000гр',
 '1000грн',
 '1000шт',
 '1001',
 '1002',
 '100500',
 '100ml',
 '100г',
 '100гр',
 '100грн',
 '100м',
 '100мл',
 '100мм',
 '100см',
 '100шт',
 '101',
 '102',
 '1020',
 '103',
 '1030',
 '104',
 '10400',
 '105',
 '1050',
 '105грн',
 '106',
 '1060',
 '107',
 '108',
 '1080',
 '1080p',
 '109',
 '10w',
 '10w40',
 '10а',
 '10г',
 '10го',
 '10грн',
 '10дней',
 '10кг',
 '10л',
 '10лет',
 '10м',
 '10мин',
 '10минут',
 '10мл',
 '10мм',
 '10см',
 '10ти',
 '10хв',
 '10шт',
 '11',
 '110',
 '1100',
 '11000',
 '1101',
 '1102',
 '1108',
 '110грн',
 '110см',
 '111',
 '112',
 '113',
 '114',
 '115',
 '1150',
 '116',
 '117',
 '118',
 '119',
 '12',
 '120',
 '1200',
